In [1]:
import pandas as pd

# Replace 'your_file.json' with the path to your JSON file
df = pd.read_csv(r"C:\Users\ASUS\Downloads\SC4001 Neural Network project\IMDB Dataset.csv")
print(len(df))
print(df.head())
df["sentiment"] = df["sentiment"].apply(lambda x: 1 if x == "positive" else 0)
df = df.reset_index(drop=True)




50000
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [2]:
# import numpy as np
# print(np.__version__)

In [3]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import torch

# # Load tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained("CEBaB/lstm.CEBaB.sa.5-class.exclusive.seed_42")
# model = AutoModelForSequenceClassification.from_pretrained("CEBaB/lstm.CEBaB.sa.5-class.exclusive.seed_42")

# # Prepare input text
# sampled = df.loc[:10]
# for row in sampled:
#     text = row["review"]

#     inputs = tokenizer(text, return_tensors="pt")

#     # Perform inference
#     with torch.no_grad():
#         logits = model(**inputs).logits

#     # Get predicted class
#     predicted_class = torch.argmax(logits, dim=-1).item()
#     print(f"Predicted label: {predicted_class} ground-truth: {row['sentiment']}")


In [4]:
# import pandas as pd
# import torch
# # from torchtext.models import RobertaClassificationHead, LSTMClassifier
# # from torchtext.functional import to_tensor

# # from torchtext.models import LSTMClassificationHead, LSTMEncoder
# from torchtext.models import RNNModel

# # Load pretrained RNN model
# bundle = torchtext.models.RNNModel.from_pretrained("lstm-imdb")
# print(bundle.model)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = bundle.to(device)

# # Evaluate sentiment of a sentence
# sampled = df.loc[:10]
# for row in sampled:
#     text = row["review"]
#     input_batch = bundle.transform(text)

#     with torch.no_grad():
#         prediction = model(input_batch)
#         label = torch.argmax(prediction, dim=1).item()


#     print(f"Predicted label: {label} ground-truth: {row['sentiment']}")

In [5]:
from sklearn.model_selection import train_test_split

# First split off test set (15%)
train_val_df, test_df = train_test_split(df, test_size=0.98, random_state=42, stratify=df["sentiment"])

# Then split train/val (17.65% of remaining data -> 15% of total for val)
train_df, val_df = train_test_split(train_val_df, test_size=0.5, random_state=42, stratify=train_val_df["sentiment"])

print(f"Train size: {len(train_df)}")
print(f"Validation size: {len(val_df)}")
print(f"Test size: {len(test_df)}")

Train size: 500
Validation size: 500
Test size: 49000


In [6]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from torchtext.data.utils import get_tokenizer
from collections import Counter
# from torchtext.vocab import vocab as build_vocab

# ---------------------------
# 1. Dataset preparation
# ---------------------------

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, vocab, tokenizer):
        self.texts = list(texts)
        self.labels = list(labels)
        self.vocab = vocab
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        tokens = self.tokenizer(self.texts[idx])
        indices = torch.tensor([self.vocab[token] for token in tokens], dtype=torch.long)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return indices, label

from torchtext.vocab import Vocab
from torchtext.data.utils import get_tokenizer
from collections import Counter

tokenizer = get_tokenizer("basic_english")

# Count tokens
counter = Counter()
for text in train_df["review"]:
    counter.update(tokenizer(text))

# Define special tokens
specials = ["<pad>", "<unk>"]

# Build vocab
vocab = Vocab(counter, specials=specials)

# --- Older torchtext doesn't support .set_default_index(), so define manually ---
UNK_IDX = vocab.stoi["<unk>"]

# Create a safe lookup wrapper
def safe_lookup(token):
    return vocab.stoi.get(token, UNK_IDX)

# Example usage:
print(f"Vocab size: {len(vocab)}")
print(f"Index of 'movie': {safe_lookup('movie')}, unknown token -> {safe_lookup('xyzqwe')}")

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

train_dataset = SentimentDataset(train_df["review"], train_df["sentiment"], vocab, tokenizer)
val_dataset = SentimentDataset(val_df["review"], val_df["sentiment"], vocab, tokenizer)

print(f"Train size: {len(train_dataset)}")
print(f"Val size: {len(val_dataset)}")


Vocab size: 13014
Index of 'movie': 18, unknown token -> 1
Train size: 500
Val size: 500


In [7]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [5]:

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    sequences, labels = zip(*batch)
    sequences_padded = pad_sequence(sequences, batch_first=True, padding_value=vocab["<pad>"])
    return sequences_padded, torch.tensor(labels)

class SimpleRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size, output_size, num_layers=1):
        super(SimpleRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn = nn.RNN(embed_dim, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()


    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :]) # last hidden state
        return self.sigmoid(out)

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0, verbose=False):
        self.patience = patience
        self.min_delta = min_delta
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            if self.verbose:
                print(f"Initial val loss: {val_loss:.4f}")
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            if self.verbose:
                print(f"Validation loss improved ({self.best_loss:.4f} -> {val_loss:.4f})")
            self.best_loss = val_loss
            self.counter = 0

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)

vocab_size = len(vocab)
model = SimpleRNN(vocab_size=vocab_size, embed_dim=64, hidden_size=128, output_size=1)
model.to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
early_stopping = EarlyStopping(patience=5, verbose=True)

num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.float().to(device)
        optimizer.zero_grad()
        outputs = model(inputs).squeeze(1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.float().to(device)
            outputs = model(inputs).squeeze(1)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

    early_stopping(avg_val_loss)
    if early_stopping.early_stop:
        print(f"Early stopping triggered at epoch {epoch+1}")
        break

Epoch [1/100], Train Loss: 0.7233, Val Loss: 0.7089
Initial val loss: 0.7089
Epoch [2/100], Train Loss: 0.6836, Val Loss: 0.7388
EarlyStopping counter: 1 out of 5
Epoch [3/100], Train Loss: 0.7015, Val Loss: 0.7048
Validation loss improved (0.7089 -> 0.7048)
Epoch [4/100], Train Loss: 0.6948, Val Loss: 0.7057
EarlyStopping counter: 1 out of 5
Epoch [5/100], Train Loss: 0.6993, Val Loss: 0.7095
EarlyStopping counter: 2 out of 5
Epoch [6/100], Train Loss: 0.7044, Val Loss: 0.6833
Validation loss improved (0.7048 -> 0.6833)
Epoch [7/100], Train Loss: 0.6869, Val Loss: 0.7026
EarlyStopping counter: 1 out of 5
Epoch [8/100], Train Loss: 0.6964, Val Loss: 0.6827
Validation loss improved (0.6833 -> 0.6827)
Epoch [9/100], Train Loss: 0.7061, Val Loss: 0.7062
EarlyStopping counter: 1 out of 5
Epoch [10/100], Train Loss: 0.7035, Val Loss: 0.6935
EarlyStopping counter: 2 out of 5
Epoch [11/100], Train Loss: 0.7058, Val Loss: 0.6909
EarlyStopping counter: 3 out of 5
Epoch [12/100], Train Loss: 0.7

In [7]:
model.eval()  # Set model to eval mode

correct = 0
total = 0

testdf = test_df.sample(200, random_state=42)
# testdf = test_df

for _, row in testdf.iterrows():
    test_text = row["review"]  # or appropriate text column name
    true_label = row["sentiment"]

    # Tokenize and convert to indices
    test_tokens = tokenizer(test_text)
    test_indices = torch.tensor([vocab[token] for token in test_tokens], dtype=torch.long).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(test_indices)
        # Assuming output is raw logits for binary class, apply sigmoid then threshold
        pred_prob = torch.sigmoid(output).item()  
        pred_label = 1 if pred_prob > 0.5 else 0

    total += 1
    correct += (pred_label == true_label)

accuracy = correct / total * 100
print(f"Accuracy on test set: {accuracy:.2f}%")

Accuracy on test set: 45.00%


In [ ]:
# RNN_model.save
RNN_save_path = "RNN_model.pth"

# Save model state_dict (recommended)
torch.save(RNN_model.state_dict(), RNN_save_path)
print(f"Model saved to {RNN_save_path}")

Model saved to RNN_model.pth


In [ ]:
RNN_model = SimpleRNN(vocab_size=len(vocab), embed_dim=64, hidden_size=128, output_size=1)

RNN_model.load_state_dict(torch.load(RNN_save_path))

In [8]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Initialize hidden and cell states with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # last time step hidden state
        return out

# Example instantiation
# model = LSTMModel(input_size=10, hidden_size=20, num_layers=2, output_size=1)

In [30]:
def collate_fn(batch):
    sequences, labels = zip(*batch)
    sequences_padded = pad_sequence(sequences, batch_first=True, padding_value=vocab["<pad>"])
    return torch.tensor(sequences_padded, dtype=torch.long), torch.tensor(labels, dtype=torch.float32)

# ----------------------------------
# 2. LSTM model (with embedding)
# ----------------------------------
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=vocab["<pad>"])
        self.lstm = nn.LSTM(embed_dim, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        # self.sigmoid = nn.Sigmoid()  # optional if using BCELoss

    def forward(self, x):
        x = self.embedding(x)
        h0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size, device=x.device)
        c0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size, device=x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # last timestep output
        return out  # logits (use BCEWithLogitsLoss)

# ----------------------------------
# 3. Early stopping (same as yours)
# ----------------------------------
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0, verbose=False):
        self.patience = patience
        self.min_delta = min_delta
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} / {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            if self.verbose:
                print(f"Validation loss improved: {self.best_loss:.4f} → {val_loss:.4f}")
            self.best_loss = val_loss
            self.counter = 0

# ----------------------------------
# 4. Setup
# ----------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)

vocab_size = len(vocab)
LSTM_model = LSTMModel(vocab_size=vocab_size, embed_dim=64, hidden_size=128, num_layers=2, output_size=1)
LSTM_model.to(device)

criterion = nn.BCEWithLogitsLoss()  # stable version
optimizer = optim.Adam(LSTM_model.parameters(), lr=0.001)
early_stopping = EarlyStopping(patience=5, verbose=True)

# ----------------------------------
# 5. Training loop
# ----------------------------------
num_epochs = 10

for epoch in range(num_epochs):
    LSTM_model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = LSTM_model(inputs).squeeze(1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)

    # Validation
    LSTM_model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = LSTM_model(inputs).squeeze(1)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

    # early_stopping(avg_val_loss)
    # if early_stopping.early_stop:
    #     print(f"Early stopping triggered at epoch {epoch+1}")
    #     break

C:\Users\ASUS\AppData\Local\Temp\ipykernel_46936\2815759266.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(sequences_padded, dtype=torch.long), torch.tensor(labels, dtype=torch.float32)


Epoch [1/10] - Train Loss: 0.6961 | Val Loss: 0.6935
Epoch [2/10] - Train Loss: 0.6874 | Val Loss: 0.6946
Epoch [3/10] - Train Loss: 0.7004 | Val Loss: 0.7019
Epoch [4/10] - Train Loss: 0.6366 | Val Loss: 0.7331
Epoch [5/10] - Train Loss: 0.5762 | Val Loss: 0.8025
Epoch [6/10] - Train Loss: 0.7024 | Val Loss: 0.6931
Epoch [7/10] - Train Loss: 0.6908 | Val Loss: 0.6971
Epoch [8/10] - Train Loss: 0.6509 | Val Loss: 0.7606
Epoch [9/10] - Train Loss: 0.5813 | Val Loss: 0.7867
Epoch [10/10] - Train Loss: 0.5576 | Val Loss: 0.7850


In [31]:
LSTM_model.eval()
correct = 0
total = 0

# for _, row in test_df.sample(400).iterrows():
for _, row in test_df.iterrows():
    text = row["review"]
    true_label = int(row["sentiment"])  # or map if it's a string

    tokens = tokenizer(text)
    indices = torch.tensor([vocab[token] for token in tokens], dtype=torch.long).unsqueeze(0).to(device)

    with torch.no_grad():
        output = LSTM_model(indices)
        pred_prob = torch.sigmoid(output).item()
        pred_label = 1 if pred_prob > 0.5 else 0

    correct += (pred_label == true_label)
    total += 1

accuracy = correct / total * 100
print(f"Accuracy on test set: {accuracy:.2f}%")


Accuracy on test set: 55.20%


In [ ]:
LSTM_save_path = "LSTM_model.pth"

# Save model state_dict (recommended)
torch.save(LSTM_model.state_dict(), LSTM_save_path)
print(f"Model saved to {LSTM_save_path}")

Model saved to LSTM_model.pth


In [ ]:
LSTM_model = LSTMModel(vocab_size=vocab_size, embed_dim=64, hidden_size=128, num_layers=2, output_size=1)
LSTM_model.to(device)

LSTM_model.load_state_dict(torch.load(LSTM_save_path))

In [13]:
from gensim.models import Word2Vec
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
import torch

tokenizer = get_tokenizer("basic_english")
corpus_tokens = [tokenizer(text) for text in train_df["review"]]

# Train Word2Vec model (or load pretrained)
w2v_model = Word2Vec(sentences=corpus_tokens, vector_size=100, window=5, min_count=1, workers=4)

w2v_vocab = w2v_model.wv.key_to_index  
UNK_TOKEN = "<unk>"

orig_vocab_size = len(w2v_vocab)
unk_index = orig_vocab_size  

new_vocab = dict(w2v_vocab)
new_vocab[UNK_TOKEN] = unk_index

embedding_dim = w2v_model.vector_size
vocab_size = orig_vocab_size + 1  # one extra for <unk>

# Initialize embedding matrix with extra row
embedding_matrix = torch.zeros((vocab_size, embedding_dim))

# Fill embeddings for known words
for word, idx in w2v_vocab.items():
    embedding_matrix[idx] = torch.tensor(w2v_model.wv[word])

# Initialize <unk> vector as zeros (or random)
embedding_matrix[unk_index] = torch.zeros(embedding_dim)

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, vocab, tokenizer):
        self.texts = texts
        self.labels = labels
        self.vocab = vocab  # word2vec vocab: word -> idx
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        tokens = self.tokenizer(self.texts[idx])
        indices = []
        for token in tokens:
            if token in self.vocab:
                indices.append(self.vocab[token])
            else:
                # UNK token
                indices.append(self.vocab[UNK_TOKEN])

        indices = torch.tensor(indices, dtype=torch.long)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return indices, label

train_df = train_df.reset_index(drop=True)
train_dataset = SentimentDataset(train_df["review"], train_df["sentiment"], new_vocab, tokenizer)
print(f"train size: {len(train_dataset)}")

val_df = val_df.reset_index(drop=True)
val_dataset = SentimentDataset(val_df["review"], val_df["sentiment"], new_vocab, tokenizer)
print(f"val size: {len(val_dataset)}")


train size: 500
val size: 500


In [14]:
import torch
import torch.nn as nn
from gensim.models import Word2Vec

# 2. Define LSTM model using pretrained embeddings
class LSTMWord2Vec(nn.Module):
    def __init__(self, embedding_matrix, hidden_size, output_size, num_layers=1):
        super(LSTMWord2Vec, self).__init__()
        num_embeddings, embedding_dim = embedding_matrix.size()
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=True)  # freeze=True to keep fixed
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)  # shape: (batch, seq_len, embedding_dim)
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # last timestep
        return self.sigmoid(out)

# 3. Example instantiation
embedding_matrix = embedding_matrix.to(device)
LSTM_w2v_model = LSTMWord2Vec(embedding_matrix=embedding_matrix, hidden_size=128, output_size=1)
LSTM_w2v_model.to(device)

LSTMWord2Vec(
  (embedding): Embedding(13013, 100)
  (lstm): LSTM(100, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [15]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [16]:
import torch
print(torch.version.cuda)           # Should show CUDA version string, e.g., '12.1'
print(torch.cuda.is_available())    # Should be True
print(torch.cuda.device_count())    # Number of GPUs
print(torch.cuda.get_device_name(0))

12.1
True
1
NVIDIA GeForce RTX 3050 4GB Laptop GPU


In [19]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import accuracy_score, f1_score

def collate_fn(batch):
    sequences, labels = zip(*batch)
    sequences_padded = pad_sequence(sequences, batch_first=True, padding_value=vocab["<pad>"])
    return torch.tensor(sequences_padded, dtype=torch.long), torch.tensor(labels, dtype=torch.float32)

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0, verbose=False):
        self.patience = patience
        self.min_delta = min_delta
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} / {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            if self.verbose:
                print(f"Validation loss improved: {self.best_loss:.4f} → {val_loss:.4f}")
            self.best_loss = val_loss
            self.counter = 0

# ----------------------------------
# 4. Setup
# ----------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

criterion = nn.BCEWithLogitsLoss()  # stable version
optimizer = optim.Adam(LSTM_w2v_model.parameters(), lr=0.001)
early_stopping = EarlyStopping(patience=5, verbose=True)

# ----------------------------------
# 5. Training loop
# ----------------------------------
num_epochs = 50

for epoch in range(num_epochs):
    LSTM_w2v_model.train()
    total_loss = 0
    # for inputs, labels in train_loader:
    #     inputs, labels = inputs.to(device), labels.to(device)
    #     optimizer.zero_grad()
    #     outputs = LSTM_w2v_model(inputs).squeeze(1)
    #     loss = criterion(outputs, labels)
    #     loss.backward()
    #     optimizer.step()
        # total_loss += loss.item()
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = LSTM_w2v_model(inputs).squeeze(1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        if batch_idx % 50 == 0:
            print(f"Epoch {epoch+1} Batch {batch_idx} Loss: {loss.item():.4f}")

    avg_train_loss = total_loss / len(train_loader)

    # Validation
    LSTM_w2v_model.eval()
    val_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = LSTM_w2v_model(inputs).squeeze(1)

            loss = criterion(outputs, labels)
            val_loss += loss.item()

            probs = torch.sigmoid(outputs)  # convert logits to probabilities
            preds = (probs >= 0.5).long()  # threshold probabilities to get predicted labels

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    val_acc = accuracy_score(all_labels, all_preds)
    val_f1 = f1_score(all_labels, all_preds)

    print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.4f} | Val F1: {val_f1:.4f}")

    # early_stopping(avg_val_loss)
    # if early_stopping.early_stop:
    #     print(f"Early stopping triggered at epoch {epoch+1}")
    #     break

C:\Users\ASUS\AppData\Local\Temp\ipykernel_1100\2361058783.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(sequences_padded, dtype=torch.long), torch.tensor(labels, dtype=torch.float32)


Epoch 1 Batch 0 Loss: 0.6933
Epoch [1/50] - Train Loss: 0.6932 | Val Loss: 0.6931 | Val Acc: 0.5000 | Val F1: 0.6667
Epoch 2 Batch 0 Loss: 0.6930


C:\Users\ASUS\AppData\Local\Temp\ipykernel_1100\2361058783.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(sequences_padded, dtype=torch.long), torch.tensor(labels, dtype=torch.float32)


Epoch [2/50] - Train Loss: 0.6931 | Val Loss: 0.6931 | Val Acc: 0.5000 | Val F1: 0.6667
Epoch 3 Batch 0 Loss: 0.6932


C:\Users\ASUS\AppData\Local\Temp\ipykernel_1100\2361058783.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(sequences_padded, dtype=torch.long), torch.tensor(labels, dtype=torch.float32)


...

Epoch [48/50] - Train Loss: 0.6931 | Val Loss: 0.6931 | Val Acc: 0.5000 | Val F1: 0.6667
Epoch 49 Batch 0 Loss: 0.6931


C:\Users\ASUS\AppData\Local\Temp\ipykernel_1100\2361058783.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(sequences_padded, dtype=torch.long), torch.tensor(labels, dtype=torch.float32)


Epoch [49/50] - Train Loss: 0.6931 | Val Loss: 0.6931 | Val Acc: 0.5000 | Val F1: 0.6667
Epoch 50 Batch 0 Loss: 0.6931


C:\Users\ASUS\AppData\Local\Temp\ipykernel_1100\2361058783.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(sequences_padded, dtype=torch.long), torch.tensor(labels, dtype=torch.float32)


Epoch [50/50] - Train Loss: 0.6931 | Val Loss: 0.6931 | Val Acc: 0.5000 | Val F1: 0.6667


In [ ]:
LSTM_w2v_model.eval()
correct = 0
total = 0

for _, row in test_df.iterrows():
    text = row["review"]
    true_label = int(row["sentiment"])  # or map if it's a string

    tokens = tokenizer(text)
    indices = torch.tensor([vocab[token] for token in tokens], dtype=torch.long).unsqueeze(0).to(device)

    with torch.no_grad():
        output = LSTM_w2v_model(indices)
        pred_prob = torch.sigmoid(output).item()
        pred_label = 1 if pred_prob > 0.5 else 0

    correct += (pred_label == true_label)
    total += 1

accuracy = correct / total * 100
print(f"Accuracy on test set: {accuracy:.2f}%")


Accuracy on test set: 50.00%
